In [ ]:
import cv2, os
import matplotlib.pyplot as plt
import numpy as np
from natsort import natsorted
from PIL import Image
import glob

In [ ]:
# global function
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [ ]:
TEETH_DATASET, TEETH_PBL, TEETH_NUMBER = './dataset/original-data', './dataset/pbl-mask', './dataset/teeth-color'

In [ ]:
# flip dataset (original, pbl mask, cei mask, yolo dataset, yolo annotation)

# original dataset
for teeth in natsorted(os.listdir(TEETH_DATASET)):
    filename, extension = teeth.split('.')
    if teeth.endswith(".png"):
        image = Image.open(os.path.join(TEETH_DATASET, teeth))
        image = np.asarray(image)
        image_horizontal_clip = cv2.flip(image, 1)
        image_vertical_clip = cv2.flip(image, 0)
        image_horizontal_vertical_clip = cv2.flip(image, -1)
        Image.fromarray(image_horizontal_clip).save(os.path.join(TEETH_DATASET,f'{filename}_h_flip.{extension}'))
        Image.fromarray(image_vertical_clip).save(os.path.join(TEETH_DATASET,f'{filename}_v_flip.{extension}'))
        Image.fromarray(image_horizontal_vertical_clip).save(os.path.join(TEETH_DATASET,f'{filename}_vh_flip.{extension}'))
        
# PBL mask image
for teeth in natsorted(os.listdir(TEETH_PBL)):
    filename, extension = teeth.split('.')
    if teeth.endswith(".png"):
        image = Image.open(os.path.join(TEETH_PBL, teeth))
        image = np.asarray(image)
        image_horizontal_clip = cv2.flip(image, 1)
        image_vertical_clip = cv2.flip(image, 0)
        image_horizontal_vertical_clip = cv2.flip(image, -1)
        Image.fromarray(image_horizontal_clip).save(os.path.join(TEETH_PBL,f'{filename}_h_flip.{extension}'))
        Image.fromarray(image_vertical_clip).save(os.path.join(TEETH_PBL,f'{filename}_v_flip.{extension}'))
        Image.fromarray(image_horizontal_vertical_clip).save(os.path.join(TEETH_PBL,f'{filename}_vh_flip.{extension}'))
        
# CEJ mask image
for teeth in natsorted(os.listdir(TEETH_CEJ)):
    filename, extension = teeth.split('.')
    if teeth.endswith(".png"):
        image = Image.open(os.path.join(TEETH_CEJ, teeth))
        image = np.asarray(image)
        image_horizontal_clip = cv2.flip(image, 1)
        image_vertical_clip = cv2.flip(image, 0)
        image_horizontal_vertical_clip = cv2.flip(image, -1)
        Image.fromarray(image_horizontal_clip).save(os.path.join(TEETH_CEJ,f'{filename}_h_flip.{extension}'))
        Image.fromarray(image_vertical_clip).save(os.path.join(TEETH_CEJ,f'{filename}_v_flip.{extension}'))
        Image.fromarray(image_horizontal_vertical_clip).save(os.path.join(TEETH_CEJ,f'{filename}_vh_flip.{extension}'))
        
# yolo train dataset
for teeth in natsorted(os.listdir(YOLO_TRAIN_DATASET)):
    filename, extension = teeth.split('.')
    if teeth.endswith(".png") and 'flip' not in teeth:
        image = Image.open(os.path.join(YOLO_TRAIN_DATASET, teeth))
        image = np.asarray(image)
        image_horizontal_clip = cv2.flip(image, 1)
        image_vertical_clip = cv2.flip(image, 0)
        image_horizontal_vertical_clip = cv2.flip(image, -1)
        Image.fromarray(image_horizontal_clip).save(os.path.join(YOLO_TRAIN_DATASET,f'{filename}_h_flip.{extension}'))
        Image.fromarray(image_vertical_clip).save(os.path.join(YOLO_TRAIN_DATASET,f'{filename}_v_flip.{extension}'))
        Image.fromarray(image_horizontal_vertical_clip).save(os.path.join(YOLO_TRAIN_DATASET,f'{filename}_vh_flip.{extension}'))
        
# Yolo annotation
for label in natsorted(os.listdir(YOLO_TRAIN_ANNOTATION)):
    filename, extension = label.split('.')
    if label.endswith(".txt") and 'flip' not in label:
        # horizontal flip
        with open(os.path.join(YOLO_TRAIN_ANNOTATION, label)) as yolo_annotation:
            labels, param1, param2, param3, param4 = [], [], [], [], []
            img_height, img_width = image.shape[:2]
            lines = [yolo_param.rstrip() for yolo_param in yolo_annotation.readlines()]
            for line in lines:
                label_number, param_value = line.split(' ')[0], line.split(' ')[1:]
                x , y, width, height = [float(item) for item in param_value]
                original_xc, original_yc = float(x * img_width) , float(y * img_height)
                box_height, box_width = int(float(img_height) * float(height)), int(float(img_width) * float(width))
                x_min, y_min = int(original_xc - (box_width / 2)), int(original_yc - (box_height / 2))
                x_max, y_max = int(original_xc + (box_width / 2)), int(original_yc + (box_height / 2))
                x_min_flip , y_min_flip = img_width - x_min - box_width, y_min
                x_max_flip , y_max_clip = img_width - x_min, y_max
                xc_flip, yc_flip = int((x_min_flip+x_max_flip)/2), int((y_min_flip+y_max_clip)/2)
                labels.append(label_number)
                param1.append(xc_flip/img_width)
                param2.append(yc_flip/img_height)
                param3.append(box_width/img_width)
                param4.append(box_height/img_height)
            df = pd.DataFrame({'label': labels, 'param1': param1, 'param2': param2, 'param3': param3, 'param4': param4})
            df.to_csv(os.path.join(YOLO_TRAIN_ANNOTATION, f'{filename}_h_flip.{extension}'), index=False, header=False, sep=' ')
            
        # vertical flip
        with open(os.path.join(YOLO_TRAIN_ANNOTATION, label)) as yolo_annotation:
            labels, param1, param2, param3, param4 = [], [], [], [], []
            img_height, img_width = image.shape[:2]
            lines = [yolo_param.rstrip() for yolo_param in yolo_annotation.readlines()]
            for line in lines:
                label_number, param_value = line.split(' ')[0], line.split(' ')[1:]
                x , y, width, height = [float(item) for item in param_value]
                original_xc, original_yc = float(x * img_width) , float(y * img_height)
                box_height, box_width = int(float(img_height) * float(height)), int(float(img_width) * float(width))
                x_min, y_min = int(original_xc - (box_width / 2)), int(original_yc - (box_height / 2))
                x_max, y_max = int(original_xc + (box_width / 2)), int(original_yc + (box_height / 2))
                x_min_flip , y_min_flip = x_min , img_height - y_min - box_height
                x_max_flip , y_max_flip = x_max, img_height - y_min
                xc_flip, yc_flip = int((x_min_flip+x_max_flip)/2), int((y_min_flip+y_max_flip)/2)
                labels.append(label_number)
                param1.append(xc_flip/img_width)
                param2.append(yc_flip/img_height)
                param3.append(box_width/img_width)
                param4.append(box_height/img_height)
            df = pd.DataFrame({'label': labels, 'param1': param1, 'param2': param2, 'param3': param3, 'param4': param4})
            df.to_csv(os.path.join(YOLO_TRAIN_ANNOTATION, f'{filename}_v_flip.{extension}'), index=False, header=False, sep=' ')
        
        # horizontal and vertical flip
        with open(os.path.join(YOLO_TRAIN_ANNOTATION, label)) as yolo_annotation:
            labels, param1, param2, param3, param4 = [], [], [], [], []
            img_height, img_width = image.shape[:2]
            lines = [yolo_param.rstrip() for yolo_param in yolo_annotation.readlines()]
            for line in lines:
                label_number, param_value = line.split(' ')[0], line.split(' ')[1:]
                x , y, width, height = [float(item) for item in param_value]
                original_xc, original_yc = float(x * img_width) , float(y * img_height)
                box_height, box_width = int(float(img_height) * float(height)), int(float(img_width) * float(width))
                x_min, y_min = int(original_xc - (box_width / 2)), int(original_yc - (box_height / 2))
                x_max, y_max = int(original_xc + (box_width / 2)), int(original_yc + (box_height / 2))
                x_min_flip , y_min_flip = img_width - x_min - box_width , img_height - y_min - box_height
                x_max_flip , y_max_flip = img_width - x_min, img_height - y_min
                xc_flip, yc_flip = int((x_min_flip+x_max_flip)/2), int((y_min_flip+y_max_flip)/2)
                labels.append(label_number)
                param1.append(xc_flip/img_width)
                param2.append(yc_flip/img_height)
                param3.append(box_width/img_width)
                param4.append(box_height/img_height)
            df = pd.DataFrame({'label': labels, 'param1': param1, 'param2': param2, 'param3': param3, 'param4': param4})
            df.to_csv(os.path.join(YOLO_TRAIN_ANNOTATION, f'{filename}_vh_flip.{extension}'), index=False, header=False, sep=' ')

In [ ]:
#load and draw box for yolo flip version
IMG_NAME= '1_h_flip'
image = Image.open(f'./dataset/yolov5-dataset/images/train/{IMG_NAME}.png')
image = np.asanyarray(image)
# image = np.stack((image,)*3, axis=-1) if len(image.shape) <3 else image

with open('./dataset/yolov5-dataset/labels/train/{IMG_NAME}.txt') as label:
    img_height, img_width = image.shape[:2]
    lines = [label.rstrip() for label in label.readlines()]
    for line in lines:
        x , y, width, height = [float(item) for item in line.split(' ')[1:]]
        xc, yc = float(x * img_width) , float(y * img_height)
        box_height, box_width = round(float(img_height) * float(height),5), round(float(img_width) * float(width),5)
        x_min, y_min = int(xc - (box_width / 2)), int(yc - (box_height / 2))
        x_max, y_max = int(xc + (box_width / 2)), int(yc + (box_height / 2))
        cv2.rectangle(copy_image, (x_min, y_min),(x_max, y_max), (255,60,190), 1)
plt.figure(figsize=(20,7))
plt.imshow(copy_image, cmap = 'gray')